# René Parlange, MSc
### 📚 Machine Learning Course, PhD in Computer Science
#### 🎓 Instructor: Juan Carlos Cuevas Tello, PhD
#### 🏛 Universidad Autónoma de San Luis Potosí (UASLP)

🔗 [GitHub Repository](https://github.com/parlange)

## Bernoulli-Bernoulli Restricted Boltzmann Machines using Tensorflow

In [ ]:
import numpy as np
import tensorflow as tf
import time

# Data Preparation
Pattern = np.array([[1, 1, 0, 0, 0, 0],
                    [0, 0, 1, 1, 0, 0],
                    [0, 0, 0, 0, 1, 1]], dtype=np.float32) # Ensure float32
TrainData = Pattern

start_time = time.time()

n_visible = 6
n_hidden = 6
n_epochs = 1000
learning_rate = 0.05
batch_size = 3

# Define weights and biases
W = tf.Variable(tf.random.normal([n_visible, n_hidden], 0.01, dtype=tf.float32))
vb = tf.Variable(tf.zeros([n_visible], dtype=tf.float32))
hb = tf.Variable(tf.zeros([n_hidden], dtype=tf.float32))

# Sample function
def sample(probabilities):
    return tf.nn.relu(tf.sign(probabilities - tf.random.uniform(tf.shape(probabilities))))

# Forward and Backward pass
def forward_pass(v):
    return sample(tf.sigmoid(tf.matmul(v, W) + hb))

def backward_pass(h):
    return sample(tf.sigmoid(tf.matmul(h, tf.transpose(W)) + vb))

# Contrastive Divergence (CD)
def CD(v0):
    v0 = tf.cast(v0, tf.float32)
    h0 = forward_pass(v0)
    v1 = backward_pass(h0)
    h1 = forward_pass(v1)

    positive_grad = tf.matmul(tf.transpose(v0), h0)
    negative_grad = tf.matmul(tf.transpose(v1), h1)

    dW = (positive_grad - negative_grad) / tf.cast(tf.shape(v0)[0], dtype=tf.float32)
    dvb = tf.reduce_mean(v0 - v1, 0)
    dhb = tf.reduce_mean(h0 - h1, 0)

    return dW, dvb, dhb

# Train the model
for epoch in range(n_epochs):
    for start in range(0, len(TrainData), batch_size):
        batch = TrainData[start:start+batch_size]
        dW, dvb, dhb = CD(batch)
        W.assign_add(learning_rate * dW)
        vb.assign_add(learning_rate * dvb)
        hb.assign_add(learning_rate * dhb)

# Testing
hidden_data = forward_pass(TrainData)
reconstructed_data = backward_pass(hidden_data)

end_time = time.time()

print("Original Data:")
print(TrainData)

print("\nReconstructed Data:")
print(reconstructed_data.numpy())

match = np.array_equal(TrainData, reconstructed_data.numpy())
print("\nMatch:", match)

# Print the execution time
print("\nExecution time:", end_time - start_time, "seconds")

Original Data:
[[1. 1. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 0. 1. 1.]]

Reconstructed Data:
[[1. 1. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 0. 1. 1.]]

Match: True

Execution time: 4.108672142028809 seconds
